# [Saver] 모델 저장하고 불러오기 

## 모델 저장

###### 모델을 저장하고 불러오려면 두개의 파일이 필요

  * **Meta garph**
    - Tensorflow graph저장
    - 모든 Variables, operations, collections 저장
    - ** *.meta ** 의 확장자 파일
<p></p>
  * **Checkpoint file** 
    - binary 파일로 Weight, bias, gradients 등을 저장
    - **model.ckpt.data-00000-of-00001, model.ckpt.index** 의 2가지 파일로 저장됨
    - ** *.data** 파일은 _Training variable_ 을 가짐
    - checkpoint 파일은 **최근 상태만 기록**


###### 모델 저장 방법 
* 선언 : Weight, bias Variable 정의 직후에 선언하는것이 좋음 (그 이후에 넣으면 필요없는 Variable 까지 저장됨)
<pre><code>
saver = tf.train.Saver()
</code></pre>
* 저장 
<pre><code>
saver.save(sess, '모델저장 경로')
</code></pre>

###### tf.train.Saver.save
<pre><code>
save(
    sess,
    save_path,
    global_step=None,
    latest_filename=None,
    meta_graph_suffix='meta',
    write_meta_graph=True,
    write_state=True,
    strip_default_attrs=False,
    save_debug_info=False
)
</code></pre>


* ex) 모델 학습 1000 이후에 저장 하려면... **global_step** 파라미터를 사용
<pre><code>
saver.save(sess, 'my_test_model', global_step=1000)
</code></pre>
> 아래와 같은 모델 저장 파일이 생성됨 
  * my_test_model-1000.index
  * my_test_model-1000.meta
  * my_test_model-1000.data-00000-of-00001
  * checkpoint
* ex) 정해진 학습 횟수마다 모델 저장하려면...
<pre><code>
#max_to_keep=5 => tf.train.Saver() 선언시 기본으로 최근 5개만 기록 하게 됨
if step % 1000 == 0:
    print("step : {}. cost : {}, accuracy : {}".format(step, cost_val, acc_val))
    # 학습이 1000번 수행 될때 마다 저장하며 그래프는 저장 안함
    ckpt_path = saver.save(sess, save_file, step)
    print("Save ckpt file : ",ckpt_path)
</code></pre>

* 모델의 구조는 학습하는 동안 변경되지 않으므로 **.meta** 파일은 저장하지 않도록 하려면... **write_meta_graph=False** 파리미터를 사용
<pre><code>
saver.save(sess, 'my_test_model', global_step=1000, write_meta_graph=False)
</code></pre>

###### tf.train.Saver
<pre><code>
__init__(
    var_list=None,
    reshape=False,
    sharded=False,
    max_to_keep=5,
    keep_checkpoint_every_n_hours=10000.0,
    name=None,
    restore_sequentially=False,
    saver_def=None,
    builder=None,
    defer_build=False,
    allow_empty=False,
    write_version=tf.train.SaverDef.V2,
    pad_step_number=False,
    save_relative_paths=False,
    filename=None
)
</code></pre>


* max_to_keep => 학습시 특정 시점에 모델을 저장한 ckpt파일이 저장되는 갯수(default : 5)
* max_to_keep=None or 0 일 경우 모두 저장함


* ex) 최근 2시간동안 4개의 모델만 저장하려면...**max_to_keep, keep_checkpoint_every_n_hours** 파리미터 사용

<pre><code>
# saver 선언시 파라미터 입력
saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)
</code></pre>

* ```tf.train.Saver()``` 로 선언하면 모든 Variable을 저장함
* ```tf.train.Saver(Variable//Collection 항목)``` 로 선언하면 Variable 항목만 저장하게됨
* 저장할 Variable//Collection 항목은 list, dict 형태의 자료형을 사용가능함
<pre><code>
saver = tf.train.Saver([W1, W2, b1, b2])
</code></pre>

## 모델 읽기

* 모델을 읽어오기 위하여 2가지 작업을 해야함
  1. 네트워크 생성
    * **.meta** 파일을 불러옴
    * ```tf.train.import()``` 함수를 이용함
    * 저장된 네트워크를 불러오면 기존 네트워크에 이어서 붙게 됨
    * ```tf.reset_default_graph()``` 를 실행하여 이전에 있던 그래프를 초기화 해줘야 불러오는 네트워크를 사용함
  2. 파라미터 로딩
    * ```tf.train.Saver()``` 를 이용해서 파라미터를 로딩 
    * 가져올 Variable를 선언하고 그 값을 restore 함

###### tf.train.Saver.restore
<pre><code>
restore(
    sess,
    save_path
)
</code></pre>

###### tf.train.latest_checkpoint
<pre><code>
tf.train.latest_checkpoint(
    checkpoint_dir,
    latest_filename=None
)
</code></pre>
* ex) 저장된 네트워크를 불러오고 값을 가져옴
<pre><code>
saver = tf.train.import_meta_graph('my-model-1000.meta')
saver.restore(sess, tf.train.latest_checkpoint('./'))
</code></pre>

* restore는 ```sess.run(tf.global_variables_initializer())``` 하는 부분에서 사용하며 초기화 하는 대신 저장된 데이터를 가져온다고 생각 하면 됨

# 모델 생성 및 저장 예제

In [7]:
import tensorflow as tf

# Prepare to feed input, i.e. feed_dict and placeholders
w1 = tf.placeholder(tf.float32, name="w1")
w2 = tf.placeholder(tf.float32, name="w2")
b1 = tf.Variable(2.0,dtype=tf.float32, name="bias")
feed_dict = {'w1': 4.0, 'w2': 8.0}

# Define a test operation that we will restore
w3 = w1 + w2
w4 = tf.multiply(w3, b1, name="op_to_restore")
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Create a saver object which will save all the variables
saver = tf.train.Saver()

# Run the operation by feeding input
result = sess.run(w4, {w1:feed_dict['w1'], w2:feed_dict['w2']})
print(result)
# Prints 24 which is sum of (w1+w2)*b1

# Now, save the graph
saver.save(sess, './my_test_model/my_test_model', global_step=1000)

24.0


'./my_test_model/my_test_model-1000'

# 모델 불러와서 새로운 입력값으로 처리

* placeholder 불러오기
> graph.get_tensor_by_name()

* operation 불러오기
> graph.get_operation_by_name()

In [9]:
import tensorflow as tf

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('./my_test_model/my_test_model-1000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./my_test_model'))


# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

print (sess.run(op_to_restore,feed_dict))
#This will print 60 which is calculated 
#using new values of w1 and w2 and saved value of b1. 

INFO:tensorflow:Restoring parameters from ./my_test_model/my_test_model-1000
60.0


###### 현재 graph에서 로딩 가능한 operation 종류 확인

In [10]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

w1
w2
bias/initial_value
bias
bias/Assign
bias/read
add
op_to_restore
init
save/Const
save/SaveV2/tensor_names
save/SaveV2/shape_and_slices
save/SaveV2
save/control_dependency
save/RestoreV2/tensor_names
save/RestoreV2/shape_and_slices
save/RestoreV2
save/Assign
save/restore_all
Mul/y
Mul
w1_1
w2_1
bias_1/initial_value
bias_1
bias_1/Assign
bias_1/read
add_1
op_to_restore_1
init_1
save/Const_1
save/SaveV2_1/tensor_names
save/SaveV2_1/shape_and_slices
save/SaveV2_1
save/control_dependency_1
save/RestoreV2_1/tensor_names
save/RestoreV2_1/shape_and_slices
save/RestoreV2_1
save/Assign_1
save/Assign_2
save/restore_all_1
w1_2
w2_2
bias/initial_value_1
bias_2
bias/Assign_1
bias/read_1
add_2
op_to_restore_2
init_2
save/Const_2
save/SaveV2/tensor_names_1
save/SaveV2/shape_and_slices_1
save/SaveV2_2
save/control_dependency_2
save/RestoreV2/tensor_names_1
save/RestoreV2/shape_and_slices_1
save/RestoreV2_2
save/Assign_3
save/restore_all_2
Mul/y_1
Mul_1
w1_1_1
w2_1_1
bias_1/initial_value_1
bias_1_1


# 모델 불러오고 operation과 layer 추가

In [11]:
import tensorflow as tf

# 현재 커널의 그래프 초기화
tf.reset_default_graph()


sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('./my_test_model/my_test_model-1000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./my_test_model/'))


# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

#Add more to the current graph
add_on_op = tf.multiply(op_to_restore,2)

print (sess.run(add_on_op,feed_dict))
#This will print 120.



INFO:tensorflow:Restoring parameters from ./my_test_model/my_test_model-1000
120.0
